In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [11]:
class ShuffleLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super(ShuffleLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.W = nn.Parameter(torch.Tensor(out_features, in_features))
        self.b = nn.Parameter(torch.Tensor(out_features))
        self.init_parameters()

    def init_parameters(self):
        nn.init.normal_(self.W, std=0.1)
        nn.init.constant_(self.b, 0)

    def forward(self, x):
        print(f"Size of input tensor: {x.size()}\nSize of W tensor {self.W.size()}")
        print(f"Input tensor: {x}\nWeight tensor: {self.W}")
        
        if len(self.W.size()) > 1:
            for i, x in enumerate(self.W):
                print(x)
        # Perform the matrix multiplication
        output = torch.matmul(x, self.W.t())
        
        print(f"output: {output}")
        output = output + self.b
        return output
    

In [5]:
inputX = torch.Tensor(np.arange(1, 4, dtype=float))
print(inputX)

tensor([1., 2., 3.])


In [12]:
shuffle = ShuffleLinear(3, 4)
print(shuffle.forward(inputX))

Size of input tensor: torch.Size([3])
Size of W tensor torch.Size([4, 3])
Input tensor: tensor([1., 2., 3.])
Weight tensor: Parameter containing:
tensor([[ 0.0325,  0.0239, -0.0041],
        [ 0.0568,  0.1109,  0.1007],
        [ 0.0327,  0.0724, -0.0115],
        [-0.0549, -0.0090,  0.0653]], requires_grad=True)
tensor([ 0.0325,  0.0239, -0.0041], grad_fn=<UnbindBackward0>)
tensor([0.0568, 0.1109, 0.1007], grad_fn=<UnbindBackward0>)
tensor([ 0.0327,  0.0724, -0.0115], grad_fn=<UnbindBackward0>)
tensor([-0.0549, -0.0090,  0.0653], grad_fn=<UnbindBackward0>)
output: tensor([-0.0023,  0.0025, -0.0032,  0.0074], grad_fn=<SqueezeBackward3>)
tensor([-0.0023,  0.0025, -0.0032,  0.0074], grad_fn=<AddBackward0>)


In [13]:
sm1 = ShuffleLinear(3, 4)
sm2 = ShuffleLinear(4, 2)
sm2.forward(sm1.forward(inputX))

Size of input tensor: torch.Size([3])
Size of W tensor torch.Size([4, 3])
Input tensor: tensor([1., 2., 3.])
Weight tensor: Parameter containing:
tensor([[-0.0961,  0.1641,  0.0316],
        [ 0.0759,  0.0096, -0.0827],
        [ 0.1406,  0.0501, -0.1838],
        [-0.1202, -0.0066,  0.0109]], requires_grad=True)
tensor([-0.0961,  0.1641,  0.0316], grad_fn=<UnbindBackward0>)
tensor([ 0.0759,  0.0096, -0.0827], grad_fn=<UnbindBackward0>)
tensor([ 0.1406,  0.0501, -0.1838], grad_fn=<UnbindBackward0>)
tensor([-0.1202, -0.0066,  0.0109], grad_fn=<UnbindBackward0>)
output: tensor([ 0.0108, -0.0101, -0.0192,  0.0146], grad_fn=<SqueezeBackward3>)
Size of input tensor: torch.Size([4])
Size of W tensor torch.Size([2, 4])
Input tensor: tensor([ 0.0108, -0.0101, -0.0192,  0.0146], grad_fn=<AddBackward0>)
Weight tensor: Parameter containing:
tensor([[ 0.0958,  0.0461, -0.0107, -0.0646],
        [-0.0569, -0.0848,  0.0499, -0.0972]], requires_grad=True)
tensor([ 0.0958,  0.0461, -0.0107, -0.0646], 

tensor([-0.0036,  0.0224], grad_fn=<AddBackward0>)

In [15]:
test = nn.Parameter(torch.Tensor(3, 4, 5))
print(test)

Parameter containing:
tensor([[[4.2320e+21, 8.6276e+00, 4.6317e+27, 4.3065e+21, 7.6724e+34],
         [3.0345e+32, 4.9639e+28, 2.7946e+20, 2.0535e-19, 7.2128e+22],
         [1.3741e+22, 1.6673e+19, 6.6830e+22, 3.2483e+33, 1.9690e-19],
         [6.8589e+22, 1.3340e+31, 1.1708e-19, 7.2128e+22, 1.3741e+22]],

        [[1.6673e+19, 6.6830e+22, 6.2618e+22, 4.7428e+30, 1.1096e+27],
         [1.2102e+25, 4.9658e+28, 7.1839e+22, 2.5045e-12, 7.5555e+31],
         [4.5144e+27, 1.0084e-08, 1.8037e+28, 1.7237e+25, 1.0755e-11],
         [9.1041e-12, 6.2609e+22, 4.7428e+30, 1.8461e+20, 9.0941e-04]],

        [[2.6908e+20, 4.4367e+27, 5.1931e-11, 1.3570e-14, 1.8888e+31],
         [1.2712e+31, 3.2745e-12, 7.5555e+31, 2.7205e+23, 5.9667e-02],
         [1.2102e+25, 1.1839e+22, 4.3317e-02, 1.4586e-19, 2.0704e-19],
         [1.3819e-08, 2.5178e-12, 1.4583e-19, 1.8888e+31, 1.9421e+20]]],
       requires_grad=True)
